get the data from https://www.dekudeals.com/

this website can know the price of each game

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import os


In [5]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import concurrent.futures
import threading

# 基本設定
base_url = "https://www.dekudeals.com/games"

# 更完整的headers模擬真實瀏覽器
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.dekudeals.com/",
    "Cache-Control": "max-age=0"
}

# 用於存儲所有遊戲數據和鎖
all_games = []
lock = threading.Lock()

# 函數：爬取一頁內容
def scrape_page(page_num):
    url = f"{base_url}?page={page_num}"
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            
            # 檢查是否有遊戲卡片
            game_cards = soup.select("div.col.pb-4.d-block")
            
            page_games = []
            for card in game_cards:
                # 遊戲名稱
                name_tag = card.select_one("h6.line-clamp-2.mt-1")
                game_name = name_tag.text.strip() if name_tag else "N/A"
                
                # 原價
                original_price_tag = card.select_one("s.text-muted")
                original_price = original_price_tag.text.strip() if original_price_tag else "N/A"
                
                # 現價
                current_price_tag = card.select_one("strong")
                current_price = current_price_tag.text.strip() if current_price_tag else "N/A"
                
                # 折扣百分比
                discount_tag = card.select_one("span.badge.badge-danger")
                discount = discount_tag.text.strip() if discount_tag else "N/A"
                
                # 價格標籤
                price_label_tag = card.select_one("span.badge.badge-warning")
                price_label = price_label_tag.text.strip() if price_label_tag else "N/A"
                
                # 銷售結束日期
                sale_ends_tag = card.select_one("div.w-100")
                sale_ends = sale_ends_tag.text.strip() if sale_ends_tag else "N/A"
                
                # 遊戲鏈接
                link_tag = card.select_one("a.main-link")
                game_link = link_tag['href'] if link_tag and 'href' in link_tag.attrs else "N/A"
                
                game_data = {
                    "遊戲名稱": game_name,
                    "原價": original_price,
                    "現價": current_price,
                    "折扣": discount,
                    "價格標籤": price_label,
                    "銷售結束": sale_ends,
                    "遊戲鏈接": game_link,
                    "頁碼": page_num
                }
                
                page_games.append(game_data)
                
            return page_games
        else:
            print(f"  請求失敗，狀態碼: {response.status_code}")
            return []
    except Exception as e:
        print(f"  爬取出錯: {e}")
        return []

# 函數：嘗試找出總頁數
def find_total_pages():
    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 查找分頁信息
        pagination = soup.select("ul.pagination li a.page-link")
        if pagination:
            # 尋找所有頁碼
            page_numbers = []
            for link in pagination:
                try:
                    if link.text.strip().isdigit():
                        page_numbers.append(int(link.text.strip()))
                except:
                    pass
                    
            if page_numbers:
                return max(page_numbers)
    else:
        return response.status_code
    
# 主爬蟲函數
def main():
    # 嘗試找出總頁數
    estimated_total_pages = find_total_pages()
    print(f"預估總頁數: {estimated_total_pages}")
    
    max_worker_thread = 2
    print(f'使用 {max_worker_thread} 個執行緒')

    # 使用執行緒池進行並行爬取
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_worker_thread) as executor:
        # 提交所有頁面的爬取任務
        future_to_page = {executor.submit(scrape_page, page): page for page in range(1, estimated_total_pages + 1)}
        
        # 獲取結果
        completed_pages = 0
        for future in concurrent.futures.as_completed(future_to_page):
            page = future_to_page[future]
            try:
                # 獲取並保存爬取結果 - 這是修復的關鍵
                page_data = future.result()
                if page_data:
                    # 使用鎖保護共享資源
                    with lock:
                        all_games.extend(page_data)
                
                completed_pages += 1
                print(f"第 {page} 頁爬取完成 (進度: {completed_pages}/{estimated_total_pages})")
                
                # 適度延遲避免請求過快
                time.sleep(random.uniform(0.5, 1))
            
            except Exception as e:
                print(f"處理第 {page} 頁結果時出錯: {e}")
    
    # 將結果轉換為DataFrame
    if all_games:
        df = pd.DataFrame(all_games)
        
        # 顯示結果統計
        print("\n爬取完成！")
        print(f"總共爬取了 {len(all_games)} 個遊戲數據，共 {estimated_total_pages} 頁")
        print("\n數據預覽:")
        print(df.head())
        
        # 保存到CSV文件
        csv_filename = "dekudeals_games.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"\n數據已保存到 {csv_filename}")
    else:
        print("沒有爬取到任何遊戲數據")

# 執行主函數
if __name__ == "__main__":
    main()

預估總頁數: 566
使用 2 個執行緒
第 1 頁爬取完成 (進度: 1/566)
第 2 頁爬取完成 (進度: 2/566)
第 3 頁爬取完成 (進度: 3/566)
第 4 頁爬取完成 (進度: 4/566)
第 6 頁爬取完成 (進度: 5/566)
第 5 頁爬取完成 (進度: 6/566)
第 7 頁爬取完成 (進度: 7/566)
第 8 頁爬取完成 (進度: 8/566)
第 9 頁爬取完成 (進度: 9/566)
第 10 頁爬取完成 (進度: 10/566)
第 11 頁爬取完成 (進度: 11/566)
第 12 頁爬取完成 (進度: 12/566)
第 13 頁爬取完成 (進度: 13/566)
第 14 頁爬取完成 (進度: 14/566)
第 15 頁爬取完成 (進度: 15/566)
第 16 頁爬取完成 (進度: 16/566)
第 18 頁爬取完成 (進度: 17/566)
第 17 頁爬取完成 (進度: 18/566)
第 20 頁爬取完成 (進度: 19/566)
第 19 頁爬取完成 (進度: 20/566)
第 22 頁爬取完成 (進度: 21/566)
第 21 頁爬取完成 (進度: 22/566)
第 24 頁爬取完成 (進度: 23/566)
第 23 頁爬取完成 (進度: 24/566)
第 25 頁爬取完成 (進度: 25/566)
第 26 頁爬取完成 (進度: 26/566)
第 27 頁爬取完成 (進度: 27/566)
第 28 頁爬取完成 (進度: 28/566)
第 29 頁爬取完成 (進度: 29/566)
第 30 頁爬取完成 (進度: 30/566)
第 31 頁爬取完成 (進度: 31/566)
第 32 頁爬取完成 (進度: 32/566)
第 33 頁爬取完成 (進度: 33/566)
第 34 頁爬取完成 (進度: 34/566)
第 35 頁爬取完成 (進度: 35/566)
第 36 頁爬取完成 (進度: 36/566)
第 37 頁爬取完成 (進度: 37/566)
第 38 頁爬取完成 (進度: 38/566)
第 39 頁爬取完成 (進度: 39/566)
第 40 頁爬取完成 (進度: 40/566)
第 41 頁爬取完成 (進度: 41/566)
第 42 頁爬取完成 (進